## This is the official counterparts evaluation script of MIL tasks
* Use google colab pro+ (high RAM+GPU)
* we use the P100 GPU for the Experiments

## The code and Training process along with all record are private
* Our github page: https://github.com/sagizty/MIL-SI
* The ROSE dataset is not publicly aviliable.
* However the MICCAI 2015 chanllenge dataset is avaliable for illustration.


In [1]:
# check GPU
!nvidia-smi

Mon Aug 15 10:14:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!date --date='+8 hour'  # CST time zone

Mon Aug 15 18:14:38 UTC 2022


## Mount Google Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## create file-system enviroment
* mount your google drive first


In [4]:
# create file-system enviroment
# mount the google drive first
# https://drive.google.com/drive/u/1/my-drive

# clear colab path
!rm -rf /data
!rm -rf /home/MIL_Experiment

# create path
!mkdir /home/MIL_Experiment
!mkdir /home/MIL_Experiment/runs
!mkdir /home/MIL_Experiment/code
!mkdir /home/MIL_Experiment/saved_models
!mkdir /home/MIL_Experiment/imaging_results

!mkdir /data
!mkdir /data/MIL_Experiment
!mkdir /data/MIL_Experiment/dataset

print('Folder Tree Creation completed!')

# get the latest code from Github MIL-SI official page
!git clone https://www.github.com/sagizty/MIL-SI.git /home/MIL_Experiment/code
print('code transfer from github completed!')

# copy runs if u want to compare
# !cp -r /content/drive/MyDrive/MIL_Experiment/runs/* /home/MIL_Experiment/runs
# print('tensorboard log transfer completed!')

# copy saved_models if u want to compare
# !cp -r /content/drive/MyDrive/MIL_Experiment/saved_models/* /home/MIL_Experiment/saved_models
# print('saved_models transfer completed!')

# get the MIL and CLS dataset from github
# by its zip
# !cp /home/MIL_Experiment/code/sample_datasets/warwick_MIL.zip /data/MIL_Experiment/dataset/
!cp /home/MIL_Experiment/code/sample_datasets/warwick_CLS.zip /data/MIL_Experiment/dataset/
# unzip
# !unzip -q /data/MIL_Experiment/dataset/warwick_MIL.zip -d /data/MIL_Experiment/dataset/
!unzip -q /data/MIL_Experiment/dataset/warwick_CLS.zip -d /data/MIL_Experiment/dataset/

# alter the path
# !rm -rf /data/MIL_Experiment/dataset/warwick_MIL.zip
!rm -rf /data/MIL_Experiment/dataset/warwick_CLS.zip
print('data transfer completed!')

Folder Tree Creation completed!
Cloning into '/home/MIL_Experiment/code'...
remote: Enumerating objects: 837, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 837 (delta 16), reused 0 (delta 0), pack-reused 798
Receiving objects: 100% (837/837), 627.71 MiB | 52.12 MiB/s, done.
Resolving deltas: 100% (143/143), done.
Checking out files: 100% (624/624), done.
code transfer from github completed!
data transfer completed!


## Arrange the working enviorment

In [ ]:
# change working dir
import os
os.chdir("/home/MIL_Experiment/code")
!pwd

# get packages
!pip install -q torch==1.10.0+cu111 torchvision==0.11.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html
!pip install tensorboardX
!pip install timm==0.5.4
!pip install notifyemail
!pip install ttach


/home/MIL_Experiment/code
     |████████████▌                   | 834.1 MB 1.2 MB/s eta 0:17:25tcmalloc: large alloc 1147494400 bytes == 0x38eac000 @  0x7f7317cb7615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |███████████████▉                | 1055.7 MB 137.8 MB/s eta 0:00:08tcmalloc: large alloc 1434370048 bytes == 0x7d502000 @  0x7f7317cb7615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████            | 1336.2 MB 1.2 MB/s eta 0:11:32tcmalloc: large alloc 1792966656 bytes == 0x2334000 @  0x7f7317cb7615 0x592b76 0x

In [ ]:
!python --version

Python 3.7.13


In [ ]:
!pip list

Package                       Version
----------------------------- ----------------------------
absl-py                       1.2.0
aiohttp                       3.8.1
aiosignal                     1.2.0
alabaster                     0.7.12
albumentations                1.2.1
altair                        4.2.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.12.1
astor                         0.8.1
astropy                       4.3.1
astunparse                    1.6.3
async-timeout                 4.0.2
asynctest                     0.13.0
atari-py                      0.2.9
atomicwrites                  1.4.1
attrs                         22.1.0
audioread                     2.1.9
autograd                      1.4
Babel                         2.10.3
backcall                      0.2.0
beautifulsoup4                4.6.3
bleach                        5.0.1
blis                          0.7.

In [ ]:
!pip freeze>requirements.txt
!cp requirements.txt ../runs

## Start Training
* by command line
* use argparse to set down hyper-parameter

* 5-fold experiment is used here

# CLS counterparts

* Train

In [ ]:
!python Train.py --model_idx ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name=None, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='ViT_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 

In [ ]:
!python Train.py --model_idx ViT_384_401_PT_lf05_b4_warwick_Cutout_CLS --augmentation_name Cutout --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name='Cutout', backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='ViT_384_401_PT_lf05_b4_warwick_Cutout_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dag

In [ ]:
!python Train.py --model_idx ViT_384_401_PT_lf05_b4_warwick_CutMix_CLS --augmentation_name CutMix --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name='CutMix', backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='ViT_384_401_PT_lf05_b4_warwick_CutMix_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dag

In [ ]:
!python Train.py --model_idx ViT_384_401_PT_lf05_b4_warwick_Mixup_CLS --augmentation_name Mixup --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name='Mixup', backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='ViT_384_401_PT_lf05_b4_warwick_Mixup_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagge

In [ ]:
!python Train.py --model_idx ResNet50_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name=None, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='ResNet50_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/t

In [ ]:
!python Train.py --model_idx ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name=None, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'v

In [ ]:
!python Train.py --model_idx efficientnet_b3_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name=None, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='efficientnet_b3_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b

In [ ]:
!python Train.py --model_idx swin_b_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --batch_size 4 --lr 0.00001 --lrf 0.05 --enable_tensorboard --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
*********************************setting*************************************
Namespace(Pre_Trained_model_path=None, att_module='SimAM', attn_drop_rate=0.0, augmentation_name=None, backbone_PT_off=False, batch_size=4, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=False, enable_notify=False, enable_sam=False, enable_tensorboard=True, enable_visualize_check=False, gpu_idx=-1, intake_epochs=0, linearprobing=False, lr=1e-05, lrf=0.05, model_idx='swin_b_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, num_epochs=50, num_workers=2, opt_name='Adam', paint=True, pos_embedding_off=False)
we dont have more GPU idx here, try to use gpu_idx=0
['swin_base_patch4_window7_224',
 'swin_base_patch4_window7_224_in22k',
 'swin_base_patch4_w

# Test

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_384',
 'vit_base_patch8_224',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch16_sam_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_patch32_sam_224',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf05_b4_warwick_Cutout_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_384',
 'vit_base_patch8_224',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch16_sam_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_patch32_sam_224',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf05_b4_warwick_CutMix_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_384',
 'vit_base_patch8_224',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch16_sam_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_patch32_sam_224',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',

In [ ]:
!python Test.py --model_idx ViT_384_401_PT_lf05_b4_warwick_Mixup_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
['convit_base',
 'convit_small',
 'convit_tiny',
 'crossvit_9_240',
 'crossvit_9_dagger_240',
 'crossvit_15_240',
 'crossvit_15_dagger_240',
 'crossvit_15_dagger_408',
 'crossvit_18_240',
 'crossvit_18_dagger_240',
 'crossvit_18_dagger_408',
 'crossvit_base_240',
 'crossvit_small_240',
 'crossvit_tiny_240',
 'levit_128',
 'levit_128s',
 'levit_192',
 'levit_256',
 'levit_384',
 'vit_base_patch8_224',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_224',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_384',
 'vit_base_patch16_sam_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_384',
 'vit_base_patch32_sam_224',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',

In [ ]:
!python Test.py --model_idx ResNet50_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
test model output： tensor([[0.5564, 0.5691]], grad_fn=<AddmmBackward0>)
model is ready now!
model loaded
model : ResNet50_384_401_PT_lf05_b4_warwick_CLS
*********************************setting*************************************
Namespace(MIL_Stripe=False, att_module='SimAM', attn_drop_rate=0.0, batch_size=1, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=True, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='ResNet50_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, paint=True, pos_embedding_off=False)
Epoch: Test
----------
Epoch: test     test index of 5 minibatch: 1      time used: 0.16187763214111328
minibatch AVG loss: 0.22278977558016777
/home/MIL_Experiment/code/ut

In [ ]:
!python Test.py --model_idx ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
['vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224']
test model output： tensor([[-0.3735,  0.1534]], grad_fn=<AddmmBackward0>)
model is ready now!
model loaded
model : ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS
*********************************setting*************************************
Namespace(MIL_Stripe=False, att_module='SimAM', attn_drop_rate=0.0, batch_size=1, check_minibatch=5, cls_token_off=False, data_augmentation_mode=2, dataroot='/data/MIL_Experiment/dataset/warwick_CLS', draw_root='/home/MIL_Experiment/runs', drop_path_rate=0.0, drop_rate=0.0, edge_size=384, enable_attention_check=True, enable_notify=False, enable_tensorboard=False, enable_visualize_check=False, gpu_idx=0, model_idx='ResN50_ViT_384_401_PT_lf05_b4_warwick_CLS', model_path='/home/MIL_Experiment/saved_models', num_classes=0, paint=True, pos_embedding_off=False)
Epoch: Test
----------
Epoch: test     test index of 5

In [ ]:
!python Test.py --model_idx efficientnet_b3_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
['efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',
 'efficientnet_cc_b0_4e',
 'efficientnet_cc_b0_8e',
 'efficientnet_cc_b1_8e',
 'efficientnet_el',
 'efficientnet_el_pruned',
 'efficientnet_em',
 'efficientnet_es',
 'efficientnet_es_pruned',
 'efficientnet_l2',
 'efficientnet_lite0',
 'efficientnet_lite1',
 'efficientnet_lite2',
 'efficientnet_lite3',
 'efficientnet_lite4',
 'efficientnetv2_l',
 'efficientnetv2_m',
 'efficientnetv2_rw_m',
 'efficientnetv2_rw_s',
 'efficientnetv2_rw_t',
 'efficientnetv2_s',
 'efficientnetv2_xl',
 'gc_efficientnetv2_rw_t',
 'tf_efficientnet_b0',
 'tf_efficientnet_b0_ap',
 'tf_efficientnet_b0_ns',
 'tf_efficientnet_b1',
 'tf_efficientnet_b1_ap',
 'tf_efficientnet_b1_ns',

In [ ]:
!python Test.py --model_idx swin_b_384_401_PT_lf05_b4_warwick_CLS --edge_size 384 --data_augmentation_mode 2 --enable_attention_check --check_minibatch 5 --dataroot /data/MIL_Experiment/dataset/warwick_CLS --model_path /home/MIL_Experiment/saved_models --draw_root /home/MIL_Experiment/runs

class_names: ['benign', 'malignant']
['swin_base_patch4_window7_224',
 'swin_base_patch4_window7_224_in22k',
 'swin_base_patch4_window12_384',
 'swin_base_patch4_window12_384_in22k',
 'swin_large_patch4_window7_224',
 'swin_large_patch4_window7_224_in22k',
 'swin_large_patch4_window12_384',
 'swin_large_patch4_window12_384_in22k',
 'swin_small_patch4_window7_224',
 'swin_tiny_patch4_window7_224']
/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
test model output： tensor([[ 0.1083, -0.0856]], grad_fn=<AddmmBackward0>)
model is ready now!
model loaded
model : swin_b_384_401_PT_lf05_b4_warwick_CLS
*********************************setting*************************************
Namespace(MIL_Stripe=False, att_module='SimAM', attn_drop

# Synchronize files

In [ ]:
# change working dir
import os
os.chdir("/home/MIL_Experiment/code/utils")
!python check_log_json.py --draw_root /home/MIL_Experiment/runs --record_dir /home/MIL_Experiment/CSV_logs

In [ ]:
# create path on google drive
!mkdir /content/drive/MyDrive/MIL_SI_sample
# copy the results
!/bin/cp -rf /home/MIL_Experiment/* /content/drive/MyDrive/MIL_SI_sample/
print('results copy completed!')

mkdir: cannot create directory ‘/content/drive/MyDrive/MIL_SI_sample’: File exists
results copy completed!


In [ ]:
!date --date='+8 hour'  # CST time zone

Mon Aug 15 19:05:26 UTC 2022
